In [1]:
from dimacs import *
import os

In [2]:
class Node:
  def __init__(self, idx):
    self.idx = idx
    self.out = set()              # zbiór sąsiadów

  def connect_to(self, v):
    self.out.add(v)

def buildGraph(G):
    (V, L) = loadWeightedGraph(G)
    graph = [None] + [Node(i) for i in range(1, V+1)]

    for (x, y, _) in L:
      graph[x].connect_to(y)
      graph[y].connect_to(x)

    return graph

In [3]:
def checkLexBFS(G, vs):
  n = len(G)
  pi = [None] * n
  for i, v in enumerate(vs):
    pi[v] = i

  for i in range(n-1):
    for j in range(i+1, n-1):
      Ni = G[vs[i]].out
      Nj = G[vs[j]].out

      verts = [pi[v] for v in Nj - Ni if pi[v] < i]
      if verts:
        viable = [pi[v] for v in Ni - Nj]
        if not viable or min(verts) <= min(viable):
          return False
  return True

W każdym kroku wybierany jest jeden wierzchołek do przetworzenia. Na początku jest to dowolny wierzchołek, ale w dalszych krokach wybierane są te wierzchołki, które mają największą liczbę sąsiadów spośród już odwiedzonych wierzchołków.

In [4]:
def lexBFS(graph):
    n = len(graph) - 1  # indeksujemy od 1

    sets = [set(range(1, n + 1))]
    visited = []

    while sets:
        # Wybieranie i usuwanie wierzchołka z ostatniego zbioru.
        curr = sets[-1].pop()
        visited.append(curr)

        partition = []

        for existing_set in sets:
            # Znajdowanie sąsiadów bieżącego wierzchołka 'curr' w zbiorze 'existing_set'.
            reachable = {v for v in existing_set if curr in graph[v].out}
            unreachable = existing_set - reachable

            if unreachable:
                partition.append(unreachable)
            if reachable:
                partition.append(reachable)

        sets = partition

    return visited

Funkcje pozwalajace testować zaimplementowane algorytmy

In [5]:
def readSolution(graph):
    with open(graph, "r") as f:
        first_line = f.readline()
    return int(first_line.split("=")[1])

def compareResults(file_path, algorithm):
    file_solution = readSolution(file_path)
    algorithm_solution = algorithm(file_path)

    print(file_path)
    if file_solution == algorithm_solution:
        print("Test passed")
    else:
        print("Test failed")

def testAllGraphs(directory_path, algorithm):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            compareResults(file_path, algorithm)

$\color{yellow}{\text{Zadanie 1.}}$ Zaimplementuj algorytm sprawdzający, czy zadany graf G jest grafem przekątniowym.

In [6]:
def isChordal(G):
    G = buildGraph(G)
    lexBFS_result = lexBFS(G)

    n = len(lexBFS_result)
    rn = {}
        
    for v in lexBFS_result:
        rn[v] = set()
        parent = None
        # Sprawdzamy każdy wierzchołek połączony z 'v'.
        for u in G[v].out:
            if lexBFS_result.index(u) < lexBFS_result.index(v):
                rn[v].add(u)
                if parent is None or lexBFS_result.index(u) > lexBFS_result.index(parent):
                    parent = u

        if parent is not None:
            for u in rn[v]:
                # Jeśli 'u' nie jest 'parent' i nie istnieje krawędź z 'u' do 'parent', graf nie jest chordalny.
                if u != parent and parent not in G[u].out:
                    return False

    return True

In [7]:
testAllGraphs("graphs-lab5_6/chordal", isChordal)

graphs-lab5_6/chordal\AT
Test passed
graphs-lab5_6/chordal\clique100
Test passed
graphs-lab5_6/chordal\clique20
Test passed
graphs-lab5_6/chordal\clique200
Test passed
graphs-lab5_6/chordal\clique4
Test passed
graphs-lab5_6/chordal\clique5
Test passed
graphs-lab5_6/chordal\cycle3
Test passed
graphs-lab5_6/chordal\cycle4
Test passed
graphs-lab5_6/chordal\cycle6
Test passed
graphs-lab5_6/chordal\example-fig5
Test passed
graphs-lab5_6/chordal\franklin
Test passed
graphs-lab5_6/chordal\grid5x5
Test passed
graphs-lab5_6/chordal\grotzsch
Test passed
graphs-lab5_6/chordal\house
Test passed
graphs-lab5_6/chordal\interval-rnd10
Test passed
graphs-lab5_6/chordal\interval-rnd100
Test passed
graphs-lab5_6/chordal\interval-rnd20
Test passed
graphs-lab5_6/chordal\interval-rnd30
Test passed
graphs-lab5_6/chordal\interval-rnd50
Test passed
graphs-lab5_6/chordal\interval-rnd6
Test passed
graphs-lab5_6/chordal\interval-rnd8
Test passed
graphs-lab5_6/chordal\K33
Test passed
graphs-lab5_6/chordal\nonplana

$\color{yellow}{\text{Zadanie 2.}}$ Zaimplementuj algorytm znajdujący rozmiar największej kliki w G.

In [8]:
def maxClique(G):
    G = buildGraph(G)
    lexBFS_result = lexBFS(G)

    result = 0
    rn = {}

    for v in lexBFS_result:
        rn[v] = set()

        for u in G[v].out:
            if u in rn:
                rn[v].add(u)

        # Aktualizacja zmiennej 'result', jeśli rozmiar kliki dla 'v' jest większy.
        # Dodajemy 1, ponieważ 'v' także jest częścią kliki.
        result = max(result, len(rn[v]) + 1)

    return result

In [9]:
testAllGraphs("graphs-lab5_6/maxclique", maxClique)

graphs-lab5_6/maxclique\AT
Test passed
graphs-lab5_6/maxclique\clique100
Test passed
graphs-lab5_6/maxclique\clique20
Test passed
graphs-lab5_6/maxclique\clique200
Test passed
graphs-lab5_6/maxclique\clique4
Test passed
graphs-lab5_6/maxclique\clique5
Test passed
graphs-lab5_6/maxclique\cycle3
Test passed
graphs-lab5_6/maxclique\example-fig5
Test passed
graphs-lab5_6/maxclique\house
Test passed
graphs-lab5_6/maxclique\interval-rnd10
Test passed
graphs-lab5_6/maxclique\interval-rnd100
Test passed
graphs-lab5_6/maxclique\interval-rnd20
Test passed
graphs-lab5_6/maxclique\interval-rnd30
Test passed
graphs-lab5_6/maxclique\interval-rnd50
Test passed
graphs-lab5_6/maxclique\interval-rnd6
Test passed
graphs-lab5_6/maxclique\interval-rnd8
Test passed
graphs-lab5_6/maxclique\nonplanar-ex1
Test passed
graphs-lab5_6/maxclique\path
Test passed
graphs-lab5_6/maxclique\simple
Test passed
graphs-lab5_6/maxclique\simple-noninterval1
Test passed
graphs-lab5_6/maxclique\simple-noninterval2
Test passed


$\color{yellow}{\text{Zadanie 3.}}$ Zaimplementuj algorytm znajdujący optymalne (używające minimalnej liczby kolorów) kolorowanie grafu G i tym samym obliczający liczbę chromatyczną G.
$\color{red}{\text{Zakłada się, że graf jest przekątniowy!}}$ 

In [10]:
def coloring(G):
    G = buildGraph(G)
    lexBFS_result = lexBFS(G)

    # Inicjalizacja listy kolorów dla każdego wierzchołka. Początkowo wszystkie wierzchołki są niepokolorowane (0).
    color = [0] * len(G)

    for v in lexBFS_result:
        neighbors = G[v].out
        used = {color[u] for u in neighbors}

        # Wybór najmniejszego dostępnego koloru, który nie został użyty przez sąsiadów.
        c = 1
        while c in used:
            c += 1

        # Przypisanie wybranego koloru wierzchołkowi 'v'.
        color[v] = c

    return max(color)

In [11]:
testAllGraphs("graphs-lab5_6/coloring", coloring)

graphs-lab5_6/coloring\AT
Test passed
graphs-lab5_6/coloring\clique100
Test passed
graphs-lab5_6/coloring\clique20
Test passed
graphs-lab5_6/coloring\clique200
Test passed
graphs-lab5_6/coloring\clique4
Test passed
graphs-lab5_6/coloring\clique5
Test passed
graphs-lab5_6/coloring\cycle3
Test passed
graphs-lab5_6/coloring\example-fig5
Test passed
graphs-lab5_6/coloring\house
Test passed
graphs-lab5_6/coloring\interval-rnd10
Test passed
graphs-lab5_6/coloring\interval-rnd100
Test passed
graphs-lab5_6/coloring\interval-rnd20
Test passed
graphs-lab5_6/coloring\interval-rnd30
Test passed
graphs-lab5_6/coloring\interval-rnd50
Test passed
graphs-lab5_6/coloring\interval-rnd6
Test passed
graphs-lab5_6/coloring\interval-rnd8
Test passed
graphs-lab5_6/coloring\nonplanar-ex1
Test passed
graphs-lab5_6/coloring\path
Test passed
graphs-lab5_6/coloring\simple
Test passed
graphs-lab5_6/coloring\simple-noninterval1
Test passed
graphs-lab5_6/coloring\simple-noninterval2
Test passed


$\color{yellow}{\text{Zadanie 4.}}$ Zaimplementuj algorytm znajdujący rozmiar najmniejszego pokrycia wierzchołkowego w zadaym grafie G.

In [12]:
def vCover(G):
    G = buildGraph(G)
    lexBFS_result = lexBFS(G)
    lexBFS_result.reverse()

    # Inicjalizacja zbioru 'I', który będzie przechowywać wierzchołki nie należące do pokrycia.
    I = set()

    for v in lexBFS_result:
        neighbors = G[v].out

        if I.isdisjoint(neighbors):
            I.add(v)

    result = len(G) - len(I) - 1
    return result

In [13]:
testAllGraphs("graphs-lab5_6/vcover", vCover)

graphs-lab5_6/vcover\AT
Test passed
graphs-lab5_6/vcover\clique100
Test passed
graphs-lab5_6/vcover\clique20
Test passed
graphs-lab5_6/vcover\clique200
Test passed
graphs-lab5_6/vcover\clique4
Test passed
graphs-lab5_6/vcover\clique5
Test passed
graphs-lab5_6/vcover\cycle3
Test passed
graphs-lab5_6/vcover\example-fig5
Test passed
graphs-lab5_6/vcover\house
Test passed
graphs-lab5_6/vcover\interval-rnd10
Test passed
graphs-lab5_6/vcover\interval-rnd100
Test passed
graphs-lab5_6/vcover\interval-rnd20
Test passed
graphs-lab5_6/vcover\interval-rnd30
Test passed
graphs-lab5_6/vcover\interval-rnd50
Test passed
graphs-lab5_6/vcover\interval-rnd6
Test passed
graphs-lab5_6/vcover\interval-rnd8
Test passed
graphs-lab5_6/vcover\nonplanar-ex1
Test passed
graphs-lab5_6/vcover\path
Test passed
graphs-lab5_6/vcover\simple
Test passed
graphs-lab5_6/vcover\simple-noninterval1
Test passed
graphs-lab5_6/vcover\simple-noninterval2
Test passed
